In [1]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


file = 'sample'

if file=='sample':
    path = '/home/ubuntu/nbs/state_farm/data/sample/'
    batch_size=64
    nb_train_samples = 1600
    nb_validation_samples = 300
elif file=='mini_sample':
    path = '/home/ubuntu/nbs/state_farm/data/mini_sample/'
    batch_size=1
    nb_train_samples = 200
    nb_validation_samples = 20
else:
    print('Unknown file name')
    exit()

#path = '/home/ubuntu/nbs/state_farm/data/'
target_size=(224, 224)


#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

train_gen = ImageDataGenerator(rotation_range=0, 
                                     width_shift_range=0, 
                                     height_shift_range=0.1, 
                                     shear_range=0.1, 
                                     zoom_range=0.2, 
                                     channel_shift_range=10,
                                     fill_mode='nearest')

batches = train_gen.flow_from_directory(directory=path+'train_set',  
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True)

#batches = get_batches(path+'train_set', shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)

# initialize the model
initial_model = VGG16(weights="imagenet", include_top=True)




Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 800 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [2]:
#finetuning
x = Dense(batches.num_class, activation='softmax', W_regularizer=l2(0.))(initial_model.layers[-2].output)
model = Model(initial_model.input, x)

# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False

opt = Adam(lr=10e-4)

model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, epochs=15, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_regularizer=<keras.reg...)`
  from ipykernel import kernelapp as app


Epoch 1/15
25/25 [==============================] - 43s - loss: 2.9114 - acc: 0.2157 - val_loss: 2.4186 - val_acc: 0.2547
Epoch 2/15
25/25 [==============================] - 40s - loss: 1.5679 - acc: 0.4861 - val_loss: 2.0921 - val_acc: 0.3517
Epoch 3/15
25/25 [==============================] - 42s - loss: 1.1727 - acc: 0.6019 - val_loss: 1.9765 - val_acc: 0.4463
Epoch 4/15
25/25 [==============================] - 42s - loss: 0.9096 - acc: 0.7105 - val_loss: 1.9191 - val_acc: 0.4136
Epoch 5/15
25/25 [==============================] - 40s - loss: 0.7764 - acc: 0.7519 - val_loss: 1.9063 - val_acc: 0.4331
Epoch 6/15
25/25 [==============================] - 42s - loss: 0.6696 - acc: 0.7913 - val_loss: 1.8543 - val_acc: 0.4696
Epoch 7/15
25/25 [==============================] - 43s - loss: 0.5441 - acc: 0.8261 - val_loss: 2.0696 - val_acc: 0.4089
Epoch 8/15
25/25 [==============================] - 42s - loss: 0.5433 - acc: 0.8294 - val_loss: 1.8257 - val_acc: 0.4579
Epoch 9/15
25/25 [======

In [3]:
'''for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True

opt = Adam(lr=10e-5)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=batches.samples//batch_size, 
                    nb_epoch=10, validation_data=valid_batches, 
                    validation_steps=valid_batches.samples//batch_size)'''

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=12, validation_steps=4)`


Epoch 1/10
12/12 [==============================] - 39s - loss: 4.1783 - acc: 0.1863 - val_loss: 2.2778 - val_acc: 0.1308
Epoch 2/10
12/12 [==============================] - 36s - loss: 2.3202 - acc: 0.1442 - val_loss: 2.2711 - val_acc: 0.1337
Epoch 3/10
12/12 [==============================] - 38s - loss: 2.1525 - acc: 0.2007 - val_loss: 2.2328 - val_acc: 0.1542
Epoch 4/10
12/12 [==============================] - 39s - loss: 1.9529 - acc: 0.2965 - val_loss: 1.8238 - val_acc: 0.3131
Epoch 5/10
11/12 [==========================>...] - ETA: 2s - loss: 1.7663 - acc: 0.3835

KeyboardInterrupt: 

In [4]:
# predictions on new data
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples//batch_size)


Found 10 images belonging to 1 classes.


In [5]:
print(preds[:10])

[]
